In [1]:
# Import librairies
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Import datasets
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [3]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# Training_set & test_set preparation
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t', header=None)
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t', header=None)

In [7]:
training_set.head()

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [8]:
test_set.head()

,0,1,2,3
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


In [9]:
# Transform dataframes to arrays
training_set = np.array(training_set, dtype='int')
test_set = np.array(test_set, dtype='int')

In [10]:
training_set[:5]

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       [        1,         5,         3, 889751712]])

In [11]:
test_set[:5]

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       [        1,        17,         3, 875073198]])

In [12]:
# Get the number of users & films
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [13]:
print('Number of users: {}\nNumber of movies: {}'.format(nb_users, nb_movies))

Number of users: 943
Number of movies: 1682


In [14]:
# Convert datas to matrix
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[data[:, 0] == id_users, 1]
        id_ratings = data[data[:, 0] == id_users, 2]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [15]:
training_set = convert(training_set)
test_set = convert(test_set)

In [16]:
training_set[0][0]

5.0

In [17]:
test_set[0][0]

0.0

In [18]:
# Convert datas to tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [19]:
training_set[0]

tensor([5., 3., 4.,  ..., 0., 0., 0.])

In [20]:
test_set[0]

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [21]:
# Convert rates to booleans
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [22]:
training_set[0]

tensor([ 1.,  1.,  1.,  ..., -1., -1., -1.])

In [23]:
test_set[0]

tensor([-1., -1., -1.,  ..., -1., -1., -1.])

In [24]:
# Create the neurons network
class RBM():
    
    def __init__(self, nv, nh):
        '''
        nv: neurones visibles
        nh: neurones cachés
        W: poids
        a: biais pour nh
        b: biais pour nv
        '''
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
        
    def sample_h(self, x):
        '''
        x: nombre de neurones
        '''
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self, y):
        '''
        y: nombre de neurones
        '''
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def train(self, v0, vk, ph0, phk):
        '''
        v0: valeurs des neurones visibles à l'état initial
        vk: valeurs des neurones visibles après k itérations d'échantillonnage
        ph0: probabilité que les neurones cachés soit égale à 1 sachant v0
        phk: probabilité associée aux neurones cachés après k itérations 
        '''
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0-vk), 0)
        self.a += torch.sum((ph0-phk), 0)

In [25]:
# Create variables
nv = len(training_set[0])
nh = 100
batch_size = 100

In [26]:
# Instance class
rbm = RBM(nv, nh)

In [27]:
# Training
nb_epoch = 10

for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users-batch_size, batch_size):
        v0 = training_set[id_user:id_user + batch_size]
        vk = v0
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0]-vk[v0 >= 0]))
        s += 1.
    print('epoch: {} Loss: {}'.format(epoch, train_loss/s))

epoch: 1 Loss: 0.3470972776412964
epoch: 2 Loss: 0.26672613620758057
epoch: 3 Loss: 0.2519378364086151
epoch: 4 Loss: 0.25067755579948425
epoch: 5 Loss: 0.2446177750825882
epoch: 6 Loss: 0.2521412670612335
epoch: 7 Loss: 0.24822242558002472
epoch: 8 Loss: 0.2479179948568344
epoch: 9 Loss: 0.2451164722442627
epoch: 10 Loss: 0.25139549374580383


In [29]:
# Test
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user + 1]
    vt = test_set[id_user:id_user + 1]
    if len(vt[vt >= 0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt >= 0]-v[vt >= 0]))
        s += 1.
print('Loss: {}'.format(test_loss/s))

Loss: 0.22929324209690094
